## Questions and Assumptions

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

df = pd.read_csv('cleaned_data.csv')

We wanted to produce an analysis of the King Country Property Sales Data that would be useful to a number of people throughout the process of valuating a house. We will ask ourselves three different questions about the dataset based on data that homeowners, estate agents, and surveyors might have access to respectively. (As a secondary goal of our analysis, there are a number of guidelines from housing associations as to how various housing metrics affect house prices; we wanted to use these as a baseline and see whether our modelling could beat these coarse guidelines.)

Questions:
1. What is the relationship between basic size and quantity metrics and the price of a house? 
    * We imagined a homeowner coming to the estate agent with basic statistics about their house, and the estate agent being able to use our model in order to give them a decent prediction of how these metrics would affect their house price.
    * During our analysis we found that, for example the square footage of the property displayed too great a colinearity with the number of bedrooms for the two to be included as features in the same model. We had to look at these metrics individually in order to decide which was the more reliable for creating a model to predict house price. We do this with a pair plot between the size variables. 
    * We also tried a combination of the two of these features, square foot per bedroom to see if the relationship between the two was any more or less valid a model.

2. How much money could a homeowner get for their home if they spend the money to improve either its grade or condition?
    - We imagined a homeowner employing a surveyor to grade their house, and the surveyor being able to tell them whether it is worth it to spend the money on making the necessary improvements to move up a grade.
    - There are obvious caveats here, such as whether the spending will definitely guarantee an increase in the grading, but this certainty is also something that the model can give us some idea of.

3. Does recent renovation affect house prices?
    - If you have recently renovated your house, should you stay in it or sell it? (Much like the TV show Stay or Sell)
    - There are some major considerations with this question. If older houses are more expensive than newer ones in general, then a house that was renovated 60 years ago might be more expensive than a house built 10 years ago that has been recently renovated. 

## Cleaning

In our cleaning we found that there were a number of missing values for the `waterfront` and `yr_renovated` variables. `waterfront` is a variable 